# Equidad en Aprendizaje Automático - Trabajo Práctico

## Análisis del Dataset _German Credit Data_

**1º Cuatrimestre 2025**

**Escuela de Ciencia y Tecnología - Universidad Nacional de San Martín**

### Integrantes

- [Gerardo Toboso](https://github.com/Gerardo1909)
- [Gianni Bevilacqua](https://github.com/Gianni2025)
- [Javier Spina](https://github.com/jaspina-unsam)

---

### Objetivo

En este notebook analizamos un modelo de aprendizaje automático de clasificación básico o _baseline_ y una mejora en cuanto a justicia o equidad social (de ahora en más, _fairness_) respecto a los grupos representados por los atributos sensibles, en particular el género. Nos servimos de las métricas clásicas de _performance_, el análisis con matrices de confusión y el uso de metricas de _fairness_; así mismo, al final de este _notebook_, comparamos el modelo _baseline_ con el modelo mitigado.

### Elementos complementarios

A la par de este _notebook_, hemos creado un documento de texto que profundiza en aspectos que no necesitan el uso de código, por ejemplo allí se encuentra el contenido de _Datasheets for Datasets_ pertinente al _German Credit Data_, así como también los análisis, reflexiones y conclusiones complementarias a este documento.

---

### Índice

1. Preprocesamiento
2. Análisis Exploratorio
3. Modelo baseline
4. Evaluación Fairness
5. Mitigación
6. Evaluación final


In [1]:
# SEPARADOR ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# BORRAR PARA ENTREGA FINAL
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ACÁ EMPIEZA LO QUE PERTENECE AL ARCHIVO 01_preprocesamiento.ipynb
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

## Preprocesamiento de Datos

### 1. Carga de datos brutos

La fuente [UC Irvine - Machine Learning Repository](https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data) provee instrucciones sencillas para importar el dataset.

> Importante: Instalar los requisitos listados en `requirements.txt`. Se recomienda usar la versión `3.10.17` de Python para asegurar compatibilidad.


In [2]:
from ucimlrepo import fetch_ucirepo

In [3]:
statlog_german_credit_data = fetch_ucirepo(id=144)

X = statlog_german_credit_data.data.features
y = statlog_german_credit_data.data.targets

print("Dimensiones de los atributos %d filas x %d columnas\nPrimeras 5 filas:" % X.shape)
X.head()

Dimensiones de los atributos 1000 filas x 20 columnas
Primeras 5 filas:


,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201


Es imposible leer el dataset a simple vista sin procesar los nombres de los atributos y los valores de cada uno. En la página de la fuente hay una tabla que describe el mapeo correspondiente.

#### Tabla de variables

| Variable Name | Role    | Type        | Demographic     | Description                                                       | Units  | Missing Values |
|---------------|---------|-------------|------------------|-------------------------------------------------------------------|--------|----------------|
| Attribute1    | Feature | Categorical |                  | Status of existing checking account                               |        | no             |
| Attribute2    | Feature | Integer     |                  | Duration                                                          | months | no             |
| Attribute3    | Feature | Categorical |                  | Credit history                                                    |        | no             |
| Attribute4    | Feature | Categorical |                  | Purpose                                                           |        | no             |
| Attribute5    | Feature | Integer     |                  | Credit amount                                                     |        | no             |
| Attribute6    | Feature | Categorical |                  | Savings account/bonds                                             |        | no             |
| Attribute7    | Feature | Categorical | Other           | Present employment since                                          |        | no             |
| Attribute8    | Feature | Integer     |                  | Installment rate in percentage of disposable income               |        | no             |
| Attribute9    | Feature | Categorical | Marital Status  | Personal status and sex                                           |        | no             |
| Attribute10   | Feature | Categorical |                  | Other debtors / guarantors                                        |        | no             |
| Attribute11   | Feature | Integer     |                  | Present residence since                                           |        | no             |
| Attribute12   | Feature | Categorical |                  | Property                                                          |        | no             |
| Attribute13   | Feature | Integer     | Age             | Age                                                               | years  | no             |
| Attribute14   | Feature | Categorical |                  | Other installment plans                                           |        | no             |
| Attribute15   | Feature | Categorical | Other           | Housing                                                           |        | no             |
| Attribute16   | Feature | Integer     |                  | Number of existing credits at this bank                           |        | no             |
| Attribute17   | Feature | Categorical | Occupation      | Job                                                               |        | no             |
| Attribute18   | Feature | Integer     |                  | Number of people being liable to provide maintenance for          |        | no             |
| Attribute19   | Feature | Binary      |                  | Telephone                                                         |        | no             |
| Attribute20   | Feature | Binary      | Other           | Foreign worker                                                    |        | no             |
| class         | Target  | Binary      |                  | 1 = Good, 2 = Bad                                                  |        | no             |

#### Información adicional de variables

1. Attribute 1 (qualitative): **Status of existing checking account**
    - A11:      ... &lt;    0 DM
    - A12: 0 &lt;= ... &lt;  200 DM
    - A13:      ... &gt;= 200 DM / salary assignments for at least 1 year
    - A14: no checking account

2. Attribute 2 (numerical): **Duration in month**

3. Attribute 3 (qualitative): **Credit history**
    - A30: no credits taken/ all credits paid back duly
    - A31: all credits at this bank paid back duly
    - A32: existing credits paid back duly till now
    - A33: delay in paying off in the past
    - A34: critical account/  other credits existing (not at this bank)

4. Attribute 4 (qualitative): **Purpose**
    - A40: car (new)
    - A41: car (used)
    - A42: furniture/equipment
    - A43: radio/television
    - A44: domestic appliances
    - A45: repairs
    - A46: education
    - A47: (vacation - does not exist?)
    - A48: retraining
    - A49: business
    - A410: others

5. Attribute 5 (numerical): **Credit amount**

6. Attibute 6 (qualitative): **Savings account/bonds**
    - A61:          ... &lt;  100 DM
    - A62:   100 &lt;= ... &lt;  500 DM
    - A63:   500 &lt;= ... &lt; 1000 DM
    - A64:          .. &gt;= 1000 DM
    - A65:   unknown/ no savings account

7. Attribute 7 (qualitative): **Present employment since**
    - A71: unemployed
    - A72:       ... &lt; 1 year
    - A73: 1  &lt;= ... &lt; 4 years
    - A74: 4  &lt;= ... &lt; 7 years
    - A75:       .. &gt;= 7 years

8. Attribute 8 (numerical): **Installment rate in percentage of disposable income**

9. Attribute 9 (qualitative): **Personal status and sex**
    - A91: male divorced/separated
    - A92: female divorced/separated/married
    - A93: male single
    - A94: male married/widowed
    - A95: female single

10. Attribute 10 (qualitative): **Other debtors / guarantors**
    - A101: none
    - A102: co-applicant
    - A103: guarantor

11. Attribute 11 (numerical) **Present residence since**

12. Attribute 12 (qualitative): **Property**
    - A121: real estate
    - A122: if not A121, building society savings agreement/ life insurance
    - A123: if not A121/A122, car or other, not in attribute 6
    - A124: unknown / no property

13. Attribute 13 (numerical): **Age in years**

14. Attribute 14 (qualitative): **Other installment plans**
    - A141: bank
    - A142: stores
    - A143: none

15. Attribute 15 (qualitative): **Housing**
    - A151: rent
    - A152: own
    - A153: for free

16. Attribute 16 (numerical): **Number of existing credits at this bank**

17. Attribute 17 (qualitative): **Job**
    - A171: unemployed/ unskilled  - non-resident
    - A172: unskilled - resident
    - A173: skilled employee / official
    - A174: management/ self-employed/highly qualified employee/ officer

18. Attribute 18 (numerical): **Number of people being liable to provide maintenance for**

19. Attribute 19 (qualitative): **Telephone**
    - A191: none
    - A192: yes, registered under the customers name

20. Attribute 20 (qualitative): **Foreign worker**
    - A201: yes
    - A202: no

### 2. Transformación de códigos a etiquetas descriptivas

Para facilitar el análisis, transformaremos los códigos alfanuméricos a descripciones legibles utilizando
una función auxiliar que mapea estos códigos según la documentación oficial del German Credit Dataset.

In [4]:
# Cambiamos los nombres de las columnas
X.columns = [
    "checking_account",
    "duration",
    "credit_history",
    "purpose",
    "credit_amount",
    "savings_account",
    "employment_since",
    "installment_rate",
    "personal_status_sex",
    "other_debtors",
    "residence_since",
    "property",
    "age",
    "other_installment_plans",
    "housing",
    "number_credits",
    "job",
    "people_liable",
    "telephone",
    "foreign_worker"
]

# Y mapeamos los valores de cada atributo a etiquetas nuevas, legibles e interpretables
replacement_values = {
    "checking_account": {
        "A11": "< 0 DM",
        "A12": "0 <= ... < 200 DM",
        "A13": ">= 200 DM or salary assigned",
        "A14": "no account"
    },
    "credit_history": {
        "A30": "no credits / all paid",
        "A31": "paid at this bank",
        "A32": "paid duly till now",
        "A33": "delay in past",
        "A34": "critical account / other bank"
    },
    "purpose": {
        "A40": "car (new)",
        "A41": "car (used)",
        "A42": "furniture/equipment",
        "A43": "radio/TV",
        "A44": "domestic appliances",
        "A45": "repairs",
        "A46": "education",
        "A47": "vacation?",
        "A48": "retraining",
        "A49": "business",
        "A410": "others"
    },
    "savings_account": {
        "A61": "< 100 DM",
        "A62": "100 <= ... < 500 DM",
        "A63": "500 <= ... < 1000 DM",
        "A64": ">= 1000 DM",
        "A65": "unknown / no account"
    },
    "employment_since": {
        "A71": "unemployed",
        "A72": "< 1 year",
        "A73": "1 <= ... < 4 years",
        "A74": "4 <= ... < 7 years",
        "A75": ">= 7 years"
    },
    "personal_status_sex": {
        "A91": "male, divorced/separated",
        "A92": "female, div/sep/married",
        "A93": "male, single",
        "A94": "male, married/widowed",
        "A95": "female, single"
    },
    "other_debtors": {
        "A101": "none",
        "A102": "co-applicant",
        "A103": "guarantor"
    },
    "property": {
        "A121": "real estate",
        "A122": "building society / insurance",
        "A123": "car / other",
        "A124": "unknown / no property"
    },
    "other_installment_plans": {
        "A141": "bank",
        "A142": "stores",
        "A143": "none"
    },
    "housing": {
        "A151": "rent",
        "A152": "own",
        "A153": "for free"
    },
    "job": {
        "A171": "unemployed / non-resident",
        "A172": "unskilled - resident",
        "A173": "skilled / official",
        "A174": "management / self-employed / qualified"
    },
    "telephone": {
        "A191": "none",
        "A192": "yes, registered"
    },
    "foreign_worker": {
        "A201": "yes",
        "A202": "no"
    }
}
X = X.replace(replacement_values)

# Chequeamos que la operación esté OK
X.head()

,checking_account,duration,credit_history,purpose,credit_amount,savings_account,employment_since,installment_rate,personal_status_sex,other_debtors,residence_since,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker
0,< 0 DM,6,critical account / other bank,radio/TV,1169,unknown / no account,>= 7 years,4,"male, single",none,4,real estate,67,none,own,2,skilled / official,1,"yes, registered",yes
1,0 <= ... < 200 DM,48,paid duly till now,radio/TV,5951,< 100 DM,1 <= ... < 4 years,2,"female, div/sep/married",none,2,real estate,22,none,own,1,skilled / official,1,none,yes
2,no account,12,critical account / other bank,education,2096,< 100 DM,4 <= ... < 7 years,2,"male, single",none,3,real estate,49,none,own,1,unskilled - resident,2,none,yes
3,< 0 DM,42,paid duly till now,furniture/equipment,7882,< 100 DM,4 <= ... < 7 years,2,"male, single",guarantor,4,building society / insurance,45,none,for free,1,skilled / official,2,none,yes
4,< 0 DM,24,delay in past,car (new),4870,< 100 DM,1 <= ... < 4 years,3,"male, single",none,4,unknown / no property,53,none,for free,2,skilled / official,2,none,yes


Ahora los datos son mucho más interpretables. Cada columna tiene un nombre descriptivo (como "checking_account", "credit_history", etc.) y los valores son legibles en lenguaje natural en lugar de códigos crípticos.

### 3. Extracción de la variable de género

Observamos que la columna `personal_status_sex` contiene información combinada sobre el estado civil y el género del solicitante. Para facilitar nuestro análisis de equidad, extraeremos específicamente la información de género en una nueva columna.

In [5]:
print("Distribución de estado civil y género\n\n", X["personal_status_sex"].value_counts())

Distribución de estado civil y género

 personal_status_sex
male, single                548
female, div/sep/married     310
male, married/widowed        92
male, divorced/separated     50
Name: count, dtype: int64


Podemos observar que la columna `personal_status_sex` contiene cuatro categorías, donde solo una corresponde a mujeres (`female, div/sep/married`) y las otras tres a hombres con diferentes estados civiles.

Para facilitar el análisis de equidad por género, necesitamos crear una nueva columna `sex` que contenga exclusivamente la información de género (masculino o femenino).

In [6]:
X["sex"] = X["personal_status_sex"].map(
    {
        "male, divorced/separated": "male",
        "female, div/sep/married": "female",
        "male, single": "male",
        "male, married/widowed": "male",
        "female, single": "female"
    }
)

print("Distribución de género (biológico)\n\n", X['sex'].value_counts())

Distribución de género (biológico)

 sex
male      690
female    310
Name: count, dtype: int64


### 3. Etiquetas categóricas para la variable objetivo

La variable objetivo `class` actualmente está codificada como $1$ (bajo riesgo) y $2$ (alto riesgo). Para mejorar la interpretabilidad, especialmente en visualizaciones futuras, la cambiamos por la columna `target` con etiquetas descriptivas.

In [7]:
y.columns = ["target"]
y = y.replace({"target": {1: "low_risk", 2: "high_risk"}})
y.value_counts()

target   
low_risk     700
high_risk    300
Name: count, dtype: int64

### 5. Verificación de la consistencia de los datos
Antes de guardar los datos procesados, verificamos que los cambios que estuvimos haciendo no hayan roto el dataset
- Comprobamos que no hay valores faltantes (como afirma la fuente)
- Verificamos que los tipos de datos son los esperados
- Confirmamos que no se perdió ninguna columna en los cambios ejecutados

In [8]:
# Info de los atributos
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   checking_account         1000 non-null   object
 1   duration                 1000 non-null   int64 
 2   credit_history           1000 non-null   object
 3   purpose                  1000 non-null   object
 4   credit_amount            1000 non-null   int64 
 5   savings_account          1000 non-null   object
 6   employment_since         1000 non-null   object
 7   installment_rate         1000 non-null   int64 
 8   personal_status_sex      1000 non-null   object
 9   other_debtors            1000 non-null   object
 10  residence_since          1000 non-null   int64 
 11  property                 1000 non-null   object
 12  age                      1000 non-null   int64 
 13  other_installment_plans  1000 non-null   object
 14  housing                  1000 non-null   

In [9]:
# Info del target
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   target  1000 non-null   object
dtypes: object(1)
memory usage: 7.9+ KB


In [10]:
# Resumen estadístico de los atributos numéricos
X.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
duration,1000.0,20.90,12.06,4.0,12.0,18.0,24.00,72.0
credit_amount,1000.0,3271.26,2822.74,250.0,1365.5,2319.5,3972.25,18424.0
installment_rate,1000.0,2.97,1.12,1.0,2.0,3.0,4.00,4.0
residence_since,1000.0,2.84,1.10,1.0,2.0,3.0,4.00,4.0
age,1000.0,35.55,11.38,19.0,27.0,33.0,42.00,75.0
number_credits,1000.0,1.41,0.58,1.0,1.0,1.0,2.00,4.0
people_liable,1000.0,1.16,0.36,1.0,1.0,1.0,1.00,2.0


Confirmamos que:
- Tenemos 1000 observaciones para ambos `X` e `y`
- El dataframe de atributos tiene 21 columnas (las 20 originales más `sex`)
- El dataframe de target cambió el nombre de `class` a `target`
- No hay valores nulos o faltantes en ninguna columna
- Los tipos de datos son los esperados
- La descripción estadística parece razonable (sin hacer un análisis detallado)

In [11]:
# SEPARADOR ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# BORRAR PARA ENTREGA FINAL
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ACÁ EMPIEZA LO QUE PERTENECE AL ARCHIVO 02_EDA.ipynb
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [12]:
# SEPARADOR ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# BORRAR PARA ENTREGA FINAL
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ACÁ EMPIEZA LO QUE PERTENECE AL ARCHIVO 03_primer_modelo.ipynb
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [13]:
# SEPARADOR ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# BORRAR PARA ENTREGA FINAL
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ACÁ EMPIEZA LO QUE PERTENECE AL ARCHIVO 04_evaluacion_fairness.ipynb
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [14]:
# SEPARADOR ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# BORRAR PARA ENTREGA FINAL
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ACÁ EMPIEZA LO QUE PERTENECE AL ARCHIVO 05_modelo_mitigado.ipynb
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [15]:
# SEPARADOR ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# BORRAR PARA ENTREGA FINAL
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ACÁ EMPIEZA LO QUE PERTENECE AL ARCHIVO 06_evaluacion_final_fairness.ipynb
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~